# Usage of Population Stability (PSI) Index in Data Quality tasks.
The PSI is a widely used statistic that measures how much a variable has shifted over time. A high PSI may alert the business to a change in the characteristics of a population. PSI is commonly used to measure the shift between control data and current data.

In [1]:
import numpy as np
import pandas as pd
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

### Download data

In [2]:
dataset = "olistbr/brazilian-ecommerce"
api.dataset_download_files(dataset, path="data/olistbr/", unzip=True)

### Read to pandas

In [3]:
payments_df = pd.read_csv("data/olistbr/olist_order_payments_dataset.csv")
payments_df.shape

(103886, 5)

In [4]:
orders_df = pd.read_csv("data/olistbr/olist_orders_dataset.csv")
orders_df.dtypes

order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object

In [5]:
payments_orders_df = payments_df.merge(orders_df, on='order_id', how='left')
payments_orders_df.shape

(103886, 12)

In [6]:
payments_orders_df.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33,0a8556ac6be836b46b3e89920d59291c,delivered,2018-04-25 22:01:49,2018-04-25 22:15:09,2018-05-02 15:20:00,2018-05-09 17:36:51,2018-05-22 00:00:00
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39,f2c7fc58a9de810828715166c672f10a,delivered,2018-06-26 11:01:38,2018-06-26 11:18:58,2018-06-28 14:18:00,2018-06-29 20:32:09,2018-07-16 00:00:00
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71,25b14b69de0b6e184ae6fe2755e478f9,delivered,2017-12-12 11:19:55,2017-12-14 09:52:34,2017-12-15 20:13:22,2017-12-18 17:24:41,2018-01-04 00:00:00
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78,7a5d8efaaa1081f800628c30d2b0728f,delivered,2017-12-06 12:04:06,2017-12-06 12:13:20,2017-12-07 20:28:28,2017-12-21 01:35:51,2018-01-04 00:00:00
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45,15fd6fb8f8312dbb4674e4518d6fa3b3,delivered,2018-05-21 13:59:17,2018-05-21 16:14:41,2018-05-22 11:46:00,2018-06-01 21:44:53,2018-06-13 00:00:00


In [7]:
payments_orders_df.dtypes

order_id                          object
payment_sequential                 int64
payment_type                      object
payment_installments               int64
payment_value                    float64
customer_id                       object
order_status                      object
order_purchase_timestamp          object
order_approved_at                 object
order_delivered_carrier_date      object
order_delivered_customer_date     object
order_estimated_delivery_date     object
dtype: object

### Cast data types

In [8]:
date_time_cols = ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']

for col in date_time_cols:
    payments_orders_df[col] = pd.to_datetime(payments_orders_df[col])
payments_orders_df.dtypes

order_id                                 object
payment_sequential                        int64
payment_type                             object
payment_installments                      int64
payment_value                           float64
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object

### Util functions

In [9]:
def calculate_psi(expected_percents: np.array, actual_percents: np.array) -> float:
    # Clip freaquencies to avoid zero division
    expected_percents = np.clip(expected_percents, a_min=0.0001, a_max=None)
    actual_percents = np.clip(actual_percents, a_min=0.0001, a_max=None)
    # Calculate PSI
    psi_value = (expected_percents - actual_percents) * np.log(expected_percents / actual_percents)
    psi_value = sum(psi_value)
    return psi_value

In [10]:
def create_equal_size_buckets(series: np.array, n_bins: int):
    breakpoints = np.arange(0, n_bins + 1) / (n_bins)
    series = pd.qcut(series, breakpoints, labels=np.linspace(1/n_bins, 1, n_bins), sort=True)
    return series

### Create control and new datasets

In [11]:
control_date = pd.to_datetime("2018-04-25 00:00:00").date()
new_date = pd.to_datetime("2018-04-26 00:00:00").date()
control_day_df = payments_orders_df[payments_orders_df.order_purchase_timestamp.dt.date == control_date]
print(control_day_df.shape)
new_day_df = payments_orders_df[payments_orders_df.order_purchase_timestamp.dt.date == new_date]
print(new_day_df.shape)

(290, 12)
(265, 12)


### Calculate quantiles

In [12]:
control_day_df['quantiles'] = pd.qcut(control_day_df.payment_value, np.linspace(0,1,11), labels=np.around(np.linspace(0.1,1,10), decimals=1))
new_day_df['quantiles'] = pd.qcut(new_day_df.payment_value, np.linspace(0,1,11), labels=np.around(np.linspace(0.1,1,10), decimals=1))

/var/folders/6m/gdj6jpvn77l18qc6s98hkzb80000gn/T/ipykernel_9845/407110813.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_day_df['quantiles'] = pd.qcut(control_day_df.payment_value, np.linspace(0,1,11), labels=np.around(np.linspace(0.1,1,10), decimals=1))
/var/folders/6m/gdj6jpvn77l18qc6s98hkzb80000gn/T/ipykernel_9845/407110813.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_day_df['quantiles'] = pd.qcut(new_day_df.payment_value, np.linspace(0,1,11), labels=np.around(np.linspace(0.1,1,10

### Calculate PSI

In [13]:
new_sum = new_day_df['payment_value'].sum()
control_sum = control_day_df['payment_value'].sum()
new_percents = new_day_df.groupby('quantiles')['payment_value'].sum() / new_sum
control_percents = control_day_df.groupby('quantiles')['payment_value'].sum() / control_sum

In [14]:
calculate_psi(control_percents.values, new_percents.values)

0.014815127394085234